In [11]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
import simpler_sf

simpler_sf.simple_salesforce()

In [12]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [13]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [14]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

100%|██████████| 1/1 [00:11<00:00, 11.35s/it]


,Id,Empresa__r.Id,Empresa__r.Name,Empresa__r.Holding__c,Empresa__r.Campa_a__c,Empresa__r.Id_Empresa__c,Empresa__r.Empleador__c,Empresa__r.poliza__c,Empresa__r.RUT_Empresa__c,Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c,Name
0,a1W8c000007c5aIEAQ,0018c00002k9KwBAAU,Aldo Conti MX,-,Aldo Conti,EMP10376,Aldo Conti MX,-,-,-,False,A-9930
1,a1W8c000007c669EAA,0018c00002k9yOCAAY,Caja de Compensación de Asignación Familiar de...,Itaú,Vida Security,EMP10377,Caja de Compensación de Asignación Familiar de...,283144,81826800-9,-,False,A-9931
2,a1W8c000007c66AEAQ,0018c00002k9yODAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10378,Itaú Corpbanca,283147,97023000-9,-,False,A-9932
3,a1W8c000007c66BEAQ,0018c00002k9yOEAAY,Itaú Corpbanca,Itaú - sin servicio,Vida Security,EMP10379,Itaú Corpbanca,283148,-,-,False,A-9933
4,a1W8c000007c66CEAQ,0018c00002k9yOFAAY,Itaú Corpbanca,Itaú,Vida Security,EMP10380,Itaú Corpbanca,283149,-,-,False,A-9934


vllamadaEmpresa.shape


In [15]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

(8232, 8)

In [16]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7912, 5)

In [17]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(8238, 10)

In [18]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [19]:
query2 = "select P_liza__c, Nombre_de_Empresa__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:11<00:00, 11.00s/it]


,P_liza__c,Nombre_de_Empresa__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,LLave,dato
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,True,a3yRN0000006mmjYAA,775173998Sin Póliza Asociada,1
1,-,Apunto Servicios De Alimentacion,-,Consalud,Holding Consalud,-,Chile,False,a3yRN0000006rG9YAI,-,1
2,-,2M Inmobiliaria Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGAYAY,-,1
3,-,360 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGBYAY,-,1
4,-,361 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGCYAY,-,1


fin empresas y comienzo de rescate de pacientes


In [20]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId":"Id Contacto"
    }
)



Pacientes_SF.shape

(1921018, 10)

In [21]:
def siPrimerCero(y):
    if len(y) <= 7:
        return "NONE"
    
    if y[0] == "0":
        y = y[1:]
        
    return y

In [22]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11580\1098969947.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")


Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

Carga de Archivo carga


In [23]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Colmena"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,rut2,DV Titular,rut1,DV B.,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento
0,15028744-8,8,15028744-8,8,AFILIADO,DALIBORR,CHAVEZ,OSSANDON,CHAVEZ OSSANDON,COTIZANTE,M,1985-08-25
1,3884613-2,2,3884613-2,2,AFILIADO,ELIANA,AENISHANSLIN,PAVEZ,NaN,COTIZANTE,F,1937-12-23
2,8254856-4,4,8254856-4,4,AFILIADO,LISSETTE,PAREDES,LUNA,NaN,COTIZANTE,F,1962-06-13
3,2638598-9,9,2638598-9,9,AFILIADO,CARLOS,BOETSCH,FORNES,NaN,COTIZANTE,M,1933-10-17
4,2638598-9,9,4351475-K,K,CARGA,LUCIA MARINA,RILEY,GOMEZ,NaN,CONYUGE,F,1941-07-13


In [24]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11580\3487171931.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


(733, 13)

In [25]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11580\980738941.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(733, 14)

In [26]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()



C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11580\3035579401.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11580\3035579401.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [27]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo

Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

In [28]:

query_pacientePrograma = "SELECT id,Paciente__c,Programas_por_Empresas__r.Empresa__c from Paciente_Programas__c where padre__c =  '" + campaña +"'"
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()

100%|██████████| 1/1 [00:04<00:00,  4.90s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c
0,0018c00002amOpnAAE,0018c00002cXclQAAS
1,0018c00002amOpsAAE,0018c00002cXclQAAS
2,0018c00002amOqCAAU,0018c00002cXclQAAS
3,0018c00002amOqSAAU,0018c00002cXclQAAS
4,0018c00002amOqTAAU,0018c00002cXclQAAS


In [29]:

query_accountContact = "SELECT id,Paciente__c,Programas_por_Empresas__r.Empresa__c from Paciente_Programas__c where padre__c =  '" + campaña +"'"
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()

100%|██████████| 1/1 [00:02<00:00,  2.81s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c
0,0018c00002amOpnAAE,0018c00002cXclQAAS
1,0018c00002amOpsAAE,0018c00002cXclQAAS
2,0018c00002amOqCAAU,0018c00002cXclQAAS
3,0018c00002amOqSAAU,0018c00002cXclQAAS
4,0018c00002amOqTAAU,0018c00002cXclQAAS


In [30]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

' if(campaña!="MetLife"):\n    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) '

Rescatamos todos los ruts Id con su empresa desde la campaña

In [31]:
pacientePrograma.shape

(1368, 2)

In [32]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",        
        "Id Contacto",
    ]
]

pacien_prog_selec = pacientePrograma[
    [
        "Paciente__c",
        "Programas_por_Empresas__r.Empresa__c"
    ]
]

Pacientes_prog_rut = pd.merge(
    left=pacien_prog_selec,
    right=Pacientes_IDrut,
    how="left",
    left_on="Paciente__c",
    right_on="Id Account Paciente",
)

Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].astype(str)
Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].str.replace("-", "")
Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].str.replace("–", "")
Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].astype(str)
Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].str.replace(".", "")
Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].str.upper()
Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].str.strip()


Pacientes_prog_rut.shape


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11580\4153785291.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_prog_rut["Rut"] = Pacientes_prog_rut["Rut"].str.replace(".", "")


(1368, 5)

In [33]:
Pacientes_archivos_conId = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_IDrut,
    how="left",
    left_on="rut",
    right_on="Rut",
)

In [34]:
Pacientes_archivos_conId.to_excel("F:/Empresas SF/reportes/paciente_conId.xlsx", index=False)
Pacientes_archivos_conId.head()

,rut2,DV Titular,rut1,DV B.,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento,rut,rutT,Rut,Id Account Paciente,Id Contacto
0,15028744-8,8,15028744-8,8,AFILIADO,Daliborr,CHAVEZ,OSSANDON,Chavez Ossandon,COTIZANTE,M,1985-08-25,150287448,150287448,150287448,0018c00002cVB0qAAG,0038c000036jJS0AAM
1,3884613-2,2,3884613-2,2,AFILIADO,Eliana,AENISHANSLIN,PAVEZ,NaN,COTIZANTE,F,1937-12-23,38846132,38846132,38846132,001RN000001hzfgYAA,003RN000003O3CrYAK
2,8254856-4,4,8254856-4,4,AFILIADO,Lissette,PAREDES,LUNA,NaN,COTIZANTE,F,1962-06-13,82548564,82548564,82548564,001RN000002b983YAA,003RN000005k36UYAQ
3,2638598-9,9,2638598-9,9,AFILIADO,Carlos,BOETSCH,FORNES,NaN,COTIZANTE,M,1933-10-17,26385989,26385989,26385989,0018c00002mVtIhAAK,0038c00003Jlg8HAAR
4,2638598-9,9,4351475-K,K,CARGA,Lucia Marina,RILEY,GOMEZ,NaN,CONYUGE,F,1941-07-13,4351475K,26385989,4351475K,0018c00002mVtIiAAK,0038c00003Jlg8IAAR


In [35]:
Pacientes_prog_rut.head()

,Paciente__c,Programas_por_Empresas__r.Empresa__c,Rut,Id Account Paciente,Id Contacto
0,0018c00002amOpnAAE,0018c00002cXclQAAS,170261348,0018c00002amOpnAAE,0038c000035CYVvAAO
1,0018c00002amOpsAAE,0018c00002cXclQAAS,187987741,0018c00002amOpsAAE,0038c000035CYW0AAO
2,0018c00002amOqCAAU,0018c00002cXclQAAS,134714816,0018c00002amOqCAAU,0038c000035CYWKAA4
3,0018c00002amOqSAAU,0018c00002cXclQAAS,47670691,0018c00002amOqSAAU,0038c000035CYWaAAO
4,0018c00002amOqTAAU,0018c00002cXclQAAS,260420356,0018c00002amOqTAAU,0038c000035CYWbAAO


In [36]:
Pacientes_Archivo.head()

,rut2,DV Titular,rut1,DV B.,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento,rut,rutT
0,15028744-8,8,15028744-8,8,AFILIADO,Daliborr,CHAVEZ,OSSANDON,Chavez Ossandon,COTIZANTE,M,1985-08-25,150287448,150287448
1,3884613-2,2,3884613-2,2,AFILIADO,Eliana,AENISHANSLIN,PAVEZ,NaN,COTIZANTE,F,1937-12-23,38846132,38846132
2,8254856-4,4,8254856-4,4,AFILIADO,Lissette,PAREDES,LUNA,NaN,COTIZANTE,F,1962-06-13,82548564,82548564
3,2638598-9,9,2638598-9,9,AFILIADO,Carlos,BOETSCH,FORNES,NaN,COTIZANTE,M,1933-10-17,26385989,26385989
4,2638598-9,9,4351475-K,K,CARGA,Lucia Marina,RILEY,GOMEZ,NaN,CONYUGE,F,1941-07-13,4351475K,26385989


aca rescatamos todos los rut que estan en el archivo, y unimos con las empresas que tienen

In [37]:
PacientesConEmpresas = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_prog_rut,
    how="left",
    left_on="rut",
    right_on="Rut",
)
PacientesConEmpresas.shape

(733, 19)

In [38]:
PacientesConEmpresas.head()

,rut2,DV Titular,rut1,DV B.,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,sexo,nacimiento,rut,rutT,Paciente__c,Programas_por_Empresas__r.Empresa__c,Rut,Id Account Paciente,Id Contacto
0,15028744-8,8,15028744-8,8,AFILIADO,Daliborr,CHAVEZ,OSSANDON,Chavez Ossandon,COTIZANTE,M,1985-08-25,150287448,150287448,0018c00002cVB0qAAG,0018c00002cXclQAAS,150287448,0018c00002cVB0qAAG,0038c000036jJS0AAM
1,3884613-2,2,3884613-2,2,AFILIADO,Eliana,AENISHANSLIN,PAVEZ,NaN,COTIZANTE,F,1937-12-23,38846132,38846132,001RN000001hzfgYAA,0018c00002cXclQAAS,38846132,001RN000001hzfgYAA,003RN000003O3CrYAK
2,8254856-4,4,8254856-4,4,AFILIADO,Lissette,PAREDES,LUNA,NaN,COTIZANTE,F,1962-06-13,82548564,82548564,001RN000002b983YAA,0018c00002cXclQAAS,82548564,001RN000002b983YAA,003RN000005k36UYAQ
3,2638598-9,9,2638598-9,9,AFILIADO,Carlos,BOETSCH,FORNES,NaN,COTIZANTE,M,1933-10-17,26385989,26385989,0018c00002mVtIhAAK,0018c00002cXclQAAS,26385989,0018c00002mVtIhAAK,0038c00003Jlg8HAAR
4,2638598-9,9,4351475-K,K,CARGA,Lucia Marina,RILEY,GOMEZ,NaN,CONYUGE,F,1941-07-13,4351475K,26385989,0018c00002mVtIiAAK,0018c00002cXclQAAS,4351475K,0018c00002mVtIiAAK,0038c00003Jlg8IAAR


In [39]:
PacientesConEmpresas.to_excel("F:/Empresas SF/reportes/pacientesqueestan.xlsx", index=False)


Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

Pacientes_Archivo.drop_duplicates(
    [
        "rut",
        "rutT",
        "LLaveEmpresa",
        "LLAveRut",
    ],
    keep="last",
)
Pacientes_Archivo.shape

In [40]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [41]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(733, 25)

In [42]:
Pacientes_con_id.head()

,rut2,DV Titular,rut1,DV B.,Tipo Carga,nombres,Apellido P.,Apellido M.,apellidos,Relación,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
0,15028744-8,8,15028744-8,8,AFILIADO,Daliborr,CHAVEZ,OSSANDON,Chavez Ossandon,COTIZANTE,...,0018c00002cVB0qAAG,150287448,Chavez Ossandon Daliborr Fernando,RUT,None,None,None,Hombre,None,0038c000036jJS0AAM
1,3884613-2,2,3884613-2,2,AFILIADO,Eliana,AENISHANSLIN,PAVEZ,NaN,COTIZANTE,...,001RN000001hzfgYAA,38846132,Eliana Aenishanslin Pavez,RUT,None,None,None,Mujer,None,003RN000003O3CrYAK
2,8254856-4,4,8254856-4,4,AFILIADO,Lissette,PAREDES,LUNA,NaN,COTIZANTE,...,001RN000002b983YAA,82548564,Lissette Paredes Luna,RUT,None,None,None,Mujer,None,003RN000005k36UYAQ
3,2638598-9,9,2638598-9,9,AFILIADO,Carlos,BOETSCH,FORNES,NaN,COTIZANTE,...,0018c00002mVtIhAAK,26385989,Carlos Boetsch Fornes,RUT,None,None,None,Hombre,None,0038c00003Jlg8HAAR
4,2638598-9,9,4351475-K,K,CARGA,Lucia Marina,RILEY,GOMEZ,NaN,CONYUGE,...,0018c00002mVtIiAAK,4351475K,Lucia Marina Riley Gomez,RUT,None,None,None,Mujer,None,0038c00003Jlg8IAAR


In [43]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):        
        return email        
    else:        
        return ""
    



In [44]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
#Subir_pacientes["Email_Seteado__c"] = ""
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )


if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
46,False,Patricia,NaN,168810377,1988-03-16,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
177,False,Despina,NaN,61486682,1948-04-10,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
178,False,Virginia,NaN,72590295,1958-09-24,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
179,False,Julio Adrian,NaN,146552625,1992-04-25,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
180,False,Priscilla,NaN,156205362,1983-10-27,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [45]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""


def agregarPaciente(listado):
    vListado = listado.split(",")
    derty = dict(
        sf.Account.create(
            # derty = sf.Account.create(
            {
                "FirstName": vListado[0],
                "LastName": vListado[1],
                "IdentificationId__pc": vListado[2],
                "RecordTypeId": vListado[3],
                "BillingCountry": vListado[4],
                "Identification_Type__c": vListado[5],
                "Pa_s_del_Paciente__c": vListado[6],
                "Cuenta_validada__c": True,
                "PersonEmail": vListado[8],
                # "PersonBirthdate": datetime.strptime(vListado[9],'%Y-%m-%d %H:%M:%S' ),
                "HealthCloudGA__Gender__pc": vListado[9],
                "Phone": vListado[10],
                "BillingCity": vListado[11],
                "BillingState": vListado[12],
                "BillingStreet": vListado[13],
                "Email_Seteado__c": bool(vListado[14]),
            }
        )
    )
    return derty["id"]

In [46]:
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
46,False,Patricia,NaN,168810377,1988-03-16,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
177,False,Despina,NaN,61486682,1948-04-10,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
178,False,Virginia,NaN,72590295,1958-09-24,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
179,False,Julio Adrian,NaN,146552625,1992-04-25,Hombre,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
180,False,Priscilla,NaN,156205362,1983-10-27,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [47]:
Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)
Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
        
    ],
    keep="last",
)
Subir_pacientes.shape

vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio


(13, 12)

In [48]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["FirstName"].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["IdentificationId__pc"].astype(str)
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["RecordTypeId"].astype(str)
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingCountry"].astype(str)
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Identification_Type__c"].astype(str)
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Cuenta_validada__c"].astype(str)

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonEmail"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," 
    
""" if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  """     
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","   
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingCity"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingState"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingStreet"].astype(str) 
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," 
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Email_Seteado__c"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  

#Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)



query2 = "select P_liza__c, Nombre_de_Empresa__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)

derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId__pc": "derty", "Email": "example@example.com"})

Subir_pacientes.head()

Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account__l.xlsx", index=False)

In [49]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

' if campaña != "MetLife":\n    pacientes_en_campaña = pd.merge(\n        left=campaña_seleccionada,\n        right=Pacientes_con_id,\n        how="left",\n        left_on="Id Account",\n        right_on="Id Account Paciente",\n    )\n    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"\n    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"\n    pacientes_en_campaña.to_excel(\n        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False\n    )\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False\n    )\n '

In [50]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

' if campaña != "MetLife1":\n    campaña_en_pacientes = pd.merge(\n        left=Pacientes_con_id,\n        right=campaña_seleccionada,\n        how="left",\n        left_on="Id Account Paciente",\n        right_on="Id Account",\n    )\n    if ("poliza" in campaña_en_pacientes.columns) == True:\n        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)\n\n    campaña_en_pacientes = campaña_en_pacientes.rename(\n        columns={"Programas_por_Empresas__c": "programas por empresa"}\n    )\n    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]\n    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[\n        "programas por empresa"\n    ]\n    campaña_en_pacientes.to_excel(\n        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False\n    ) '

In [51]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [52]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

NameError: name 'campaña_seleccionada' is not defined

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()